In [1]:
import pandas as pd
import re
from pandas.core.common import flatten
import numpy as np
import more_itertools as mit
from operator import itemgetter
from itertools import groupby
from collections import defaultdict

In [2]:
class EDSF:
    
    def __init__(self, file, sheet1, sheet2): 
        # load file
        df = pd.read_excel(io=file, sheet_name=sheet1)
        # rename colums
        df.columns = ['0', 'Knowledge Area Groups', 'KAG code', 'KA code', 'Knowledge Areas', 'KU code', 'Suggested Knowledge Units', '7', 'Mapping to CCS2012']
        # drop NaN rows
        df['0'] = df.isnull().all(axis=1).cumsum().dropna()
        
        # split dataframe on NaNs
        self.d = d = {n: df.iloc[rows][1:] for n, rows in df.groupby('0').groups.items() if (len(df.iloc[rows]) > 1)}
        
        # load KAG's, KA's and KU's for the file
        self.kags = set([list(d[key]['Knowledge Area Groups'])[0] for key in self.d])
        self.kas = [list(self.d[key]['Knowledge Areas'])[0] for key in self.d]
        self.kus = [x for x in list(flatten([list(self.d[key]['Suggested Knowledge Units']) for key in self.d])) if x == x]
        
        # load codes
        self.kagcodes = [list(df['KAG code'])[0].split('/')[0] for df in self.d.values()]
        self.kacodes = [list(df['KA code'])[0] for df in self.d.values()]
        self.kucodes = [x for x in list(flatten([list(self.d[key]['KU code']) for key in self.d])) if x == x]
        
        # load competences
        df_comp = pd.read_excel(io=file, sheet_name=sheet2)
        df_comp.columns = df_comp.iloc[0]
        
        # split dataframe on knowledge area group
        idx_comp_kags = [list(df_comp.columns).index(item) for item in list(df_comp.columns) if '(' in item]
        splitted_df = []
        for idx, i in enumerate(idx_comp_kags):
            if idx != len(idx_comp_kags) - 1:
                l = list(range(i, idx_comp_kags[idx+1]))
            else:
                l = list(range(i, len(df_comp.columns)))
            splitted_df.append(df_comp.iloc[:,l].reset_index(drop=True))
        
        # split resulting dataframe on knowledge area
        self.splitted_df = []
        for df in splitted_df:
            # find the idx with nans
            boolnanlist = df.iloc[:,[1,2]].isnull()
            allnanlist = list(np.where(boolnanlist.all(axis=1))[0])

            # with multiple consecutive nans, only keep the highest idx
            nanlist = [max(list(map(itemgetter(1), g))) for k, g in groupby(enumerate(allnanlist), lambda ix: ix[0]-ix[1])]
            if not df.iloc[-1].isnull().values.all():
                nanlist = nanlist + [len(df)]

            # split the dataframe on knowledge area
            splitted = [df.iloc[nanlist[n]:nanlist[n+1]].dropna(how='all').reset_index(drop=True) for n in range(len(nanlist)-1)]
            self.splitted_df.append(splitted)
        
        # make dicts with ka and ku as key and competence as value
        self.ka_dict = defaultdict(list)
        self.ku_dict = defaultdict(list)
        for group in self.splitted_df:
            for area in group:
                for index, row in area.iloc[1:].iterrows():
                    competence = row.iloc[0]
                    ka = row.iloc[1]
                    ku = row.iloc[2]
                    ku2 = None
                    try:
                        ku2 = row.iloc[3]
                    except:
                        pass
                    if pd.notna(ka):
                        ka = ka.split(',')
                        for k in ka:
                            self.ka_dict[k.strip()].append(competence)
                    if pd.notna(ku):
                        ku = ku.split(',')
                        for k in ku:
                            self.ku_dict[k.strip()].append(competence)
                    if pd.notna(ku2):
                        ku2 = ku2.split(',')
                        for k in ku2:
                            self.ku_dict[k.strip()].append(competence)
                            
    def switch(self, input_):
        """ switch between kagcode and kacode """
        if 'k' in input_.lower():
            idx = self.kacodes.index(input_)
            return self.kagcodes[idx]
        else:
            idx = self.kagcodes.index(input_)
            return self.kacodes[idx]
        
    def get_ka_from_kag(self, kag):
        kaset = set()
        for key in self.d:
            if list(self.d[key]['Knowledge Area Groups'])[0] == kag:
                kaset.add(list(self.d[key]['Knowledge Area'])[0])
        return kaset
        
    def get_ku_from_kag(self, kag):
        kulist = []
        for key in self.d:
            if list(self.d[key]['Knowledge Area Groups'])[0] == kag:
                for item in list(self.d[key]['Suggested Knowledge Units']):
                    if item == item:
                        kulist.append(item)
        return kulist
       
    def get_kag_from_ka(self, ka):
        kagset = set()
        for key in self.d:
            if list(self.d[key]['Knowledge Areas'])[0] == ka:
                kagset.add(list(self.d[key]['Knowledge Area Groups'])[0])
        return kagset
            
    def get_ku_from_ka(self, ka):
        kulist = []
        for key in self.d:
            if list(self.d[key]['Knowledge Areas'])[0] == ka:
                for item in list(self.d[key]['Suggested Knowledge Units']):
                    if item == item:
                        kulist.append(item)
        return kulist
        
    def get_kag_from_ku(self, ku):
        kagset = set()
        for key in self.d:
            if ku in list(self.d[key]['Suggested Knowledge Units']):
                kagset.add(list(self.d[key]['Knowledge Area Groups'])[0])
        return kagset
    
    def get_ka_from_ku(self, ku):
        kaset = set()
        for key in self.d:
            if ku in list(self.d[key]['Suggested Knowledge Units']):
                kaset.add(list(self.d[key]['Knowledge Areas'])[0])
        return kaset
    
    def get_ku_from_kacode(self, code_ka):
        kulist = []
        for key in self.d:
            if list(self.d[key]['KA code'])[0].split('/')[0] == code_ka:
                for item in list(self.d[key]['Suggested Knowledge Units']):
                    if item == item:
                        kulist.append(item)
        return kulist
    
    def get_ku_from_kucode(self, code_ku):
        if code_ku[2] != '0':
            code_ku = f'{code_ku[:2]}0{code_ku[2:]}'
        idx = self.kusc.index(code_ku)
        return self.kus[idx]
    
    def get_kacode_from_ka(self, ka):
        for key in self.d:
            if list(self.d[key]['Knowledge Areas'])[0] == ka:
                return list(self.d[key]['KA code'])[0]
    
    def get_kucode_from_ku(self, ku):
        codeset = set()
        for key in self.d:
            if ku in list(self.d[key]['Suggested Knowledge Units']):
                idx = list(self.d[key]['Suggested Knowledge Units']).index(ku)
                codeset.add(list(self.d[key]['KU code'])[idx])
        return codeset

    def get_comp_from_ka(self, ka):
        code = self.get_code_from_ka(ka)
        for key in self.ka_dict:
            if key == code:
                return self.ka_dict[key]
        return None
            
    def get_comp_from_ku(self, ku):
        code = self.get_code_from_ku(ku)
        for key in list(self.ku_dict):
            mod_key = key.replace('0', '', 1)
            for cod in code:
                if mod_key == cod:
                    return self.ku_dict[key]
        return None
    
    def get_comp_from_code(self, code):
        if 'a' in code.lower():
            return self.ka_dict[code]
        elif 'u' in code.lower():
            if code[2] != '0':
                code = f'{code[:2]}0{code[2:]}'
                return self.ku_dict[code]
            else:
                return self.ku_dict[code]
        else:
            print('please input valid id')

In [3]:
obj = EDSF(file='working_dsci-dsp-competences-to-bok-v03.xlsx', 
                      sheet1='DS-BoK - release 3 (reference)',
                      sheet2='CF-DS Data Sci Maping to BoK')

In [4]:
# obj.ku_dict

In [5]:
def all_competence_with_id(obj):
    all_c = []
    for group in obj.splitted_df:
        for area in group:
            for index, row in area.iloc[1:].iterrows():
                idx = area.iloc[0][0].split('–')[0].split('-')[0].strip()
                iden = 'C_' + idx + f'.{index-1}'
                competence = row.iloc[0]
                all_c.append([iden, competence])
    return all_c

In [6]:
def all_competence_with_id2(obj):
    all_c = []
    for group in obj.splitted_df:
        for area in group:
            value = list(area.iloc[:,0])
            iden = value[0].split('–')[0].split('-')[0].strip()
            competence = ' '.join(value[1:])
            all_c.append([iden, competence])
    return all_c
            

In [7]:
all_competences = all_competence_with_id2(obj)

In [8]:
all_competences

[['DSDA',
  'Use appropriate data analytics and statistical techniques on available data to discover new relations and deliver insights into research problem or organizational processes  and support decision-making. '],
 ['DSDA01',
  'Effectively use variety of data analytics techniques, such as Machine Learning (including supervised, unsupervised, semi-supervised learning), Data Mining, Prescriptive and Predictive Analytics, for complex data analysis through the whole data lifecycle'],
 ['DSDA02',
  'Apply designated quantitative techniques, including statistics, time series analysis, optimization, and simulation to deploy appropriate models for analysis and prediction'],
 ['DSDA03',
  'Identify, extract, and pull together available and pertinent heterogeneous data, including  modern data sources such as social media data, open data, governmental data, verify data quality'],
 ['DSDA04',
  'Understand and use different performance and accuracy metrics for model validation in analytics 

In [9]:
def knowledge_units(obj):
    kalist = []
    for code in obj.kacodes:
        kus = obj.get_ku_from_kacode(code)
        flat = kus[0]
        for item in kus[1:]:
            item = item.replace('and'
                                , '')
            flat += (' ' + item)
        kalist.append([code, flat])
    return kalist

In [10]:
kagcode_with_ka = knowledge_units(obj)
kagcode_with_ka

[['KA01.01',
  'Probability & Statistics Statistical paradigms (regression, time series, dimensionality, clusters) Probabilistic representations (causal networks, Bayesian analysis, Markov nets) Frequentist  Bayesian statistics Probabilistic reasoning Exploratory  confirmatory data analysis Quantittaive analytics Qualitative Analytics Data preparation  preprocessing Performance analysis Markov models, markov networks Operations research Information theory Discrete Mathematics  Graph Theory Mathematical analysis Mathematical software  tools'],
 ['KA01.02',
  'Machine Learning theory and algorithms Supervised Machine Learning Unsupervised Machine Learning Reinforced learning Classification methods Design  Analysis of Algorithms Game Theory & Mechanism design Artificial Intelligence Statistical paradigms (regression, time series, dimensionality, clusters) Probabilistic representations (causal networks, Bayesian analysis, Markov nets) Frequentist  Bayesian statistics Probabilistic reasonin